In [1]:
import os

data_file = os.environ["DATA"] + "webinterfaces/exp02/filtered_data.pkl"


In [2]:
import pandas as pd

df = pd.read_pickle(data_file)


In [3]:
def convert_participants_df_to_within_measures_df(df):
    id_participant = []
    score = []
    reliance = []
    overreliance = []
    underreliance = []
    trust = []
    cogload = []
    xai_condition = []
    time_pressure = []
    difficulty = []
    within_condition = []

    for index, row in df.iterrows():
        for diff in ["easy", "hard"]:
            for pressure in ["mild", "strong"]:
                id_participant.append(index)
                xai_condition.append(row["xai_condition"])
                time_pressure.append(pressure)
                difficulty.append(diff)
                within_condition.append(f"{diff}_{pressure}")

                score.append(row[f"score_{diff}_{pressure}"])
                reliance.append(row[f"reliance_{diff}_{pressure}"])
                overreliance.append(row[f"overreliance_{diff}_{pressure}"])
                underreliance.append(row[f"underreliance_{diff}_{pressure}"])
                trust.append(row[f"trust_{diff}_{pressure}"])
                cogload.append(row[f"cogload_{diff}_{pressure}"])

    return pd.DataFrame({
        "participant_id": id_participant,
        "difficulty": difficulty,
        "pressure": time_pressure,
        "Difficulty/Time pressure": within_condition,
        "reliance": reliance,
        "overreliance": overreliance,
        "underreliance": underreliance,
        "score": score,
        "trust": trust,
        "cogload": cogload,
        "XAI condition": xai_condition
    })




In [4]:
from xaipatimg.analysis.dataframes import convert_participants_df_to_within_measures_df

measures_df = convert_participants_df_to_within_measures_df(df)

In [5]:
measures_df

,participant_id,difficulty,pressure,Difficulty/Time pressure,reliance,overreliance,underreliance,score,trust,cogload,XAI condition,tasks_order,task_index_order
0,5a33e4a10758280001030ce5,easy,mild,easy_mild,0.666667,0.0,0.2,0.833333,NaN,11,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",0
1,5a33e4a10758280001030ce5,easy,strong,easy_strong,0.833333,0.5,0.1,0.833333,NaN,24,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",1
2,5a33e4a10758280001030ce5,hard,mild,hard_mild,0.666667,0.0,0.2,0.833333,NaN,21,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",3
3,5a33e4a10758280001030ce5,hard,strong,hard_strong,0.416667,0.5,0.6,0.416667,NaN,29,H,"[easy_mild, easy_strong, hard_strong, hard_mild]",2
4,5bcb95ffb592580001ec24cf,easy,mild,easy_mild,0.666667,0.0,0.2,0.833333,NaN,24,H,"[hard_mild, easy_mild, easy_strong, hard_strong]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,677a8e5affbe5673b3a79e6b,hard,strong,hard_strong,0.750000,1.0,0.3,0.583333,2.0,24,H+AI+GRADCAM,"[easy_strong, hard_strong, easy_mild, hard_mild]",1
2016,5ce15b0ea7b516001828c646,easy,mild,easy_mild,0.750000,0.0,0.1,0.916667,4.0,27,H+AI+GRADCAM,"[hard_mild, hard_strong, easy_mild, easy_strong]",2
2017,5ce15b0ea7b516001828c646,easy,strong,easy_strong,0.583333,0.5,0.4,0.583333,3.0,29,H+AI+GRADCAM,"[hard_mild, hard_strong, easy_mild, easy_strong]",3
2018,5ce15b0ea7b516001828c646,hard,mild,hard_mild,0.666667,0.0,0.2,0.833333,4.0,25,H+AI+GRADCAM,"[hard_mild, hard_strong, easy_mild, easy_strong]",0


In [6]:
# df = create_df()
# df["participant"] = df["participant"].astype("category")
measures_df["pressure"] = measures_df["pressure"].astype("category")
measures_df["difficulty"] = measures_df["difficulty"].astype("category")
measures_df["XAI condition"] = measures_df["XAI condition"].astype("category")

measures_df["pressure"] = pd.Categorical(
    measures_df["pressure"],
    categories=["mild", "strong"],
    ordered=True
)

measures_df["difficulty"] = pd.Categorical(
    measures_df["difficulty"],
    categories=["easy", "hard"],
    ordered=True
)

measures_df["XAI condition"] = pd.Categorical(
    measures_df["XAI condition"],
    categories=["H", "H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)


## Test of hypotheses

### Hypotheses 1 (related to time pressure)
Each hypothesis is tested for every group with AI, against the data of the same group
with no time pressure/difficulty. Effects of pressure and
difficulty are studied independently + their interaction

* (1.a) Higher time pressure and higher difficulty increase reliance.


In [7]:
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

pvals_pressure = []
pvals_difficulty = []
pvals_both = []
labels = []
for xai_group in ["H", "H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = measures_df[measures_df["XAI condition"] == xai_group]
    print(xai_group)
    model = smf.mixedlm(
        "reliance ~ pressure * difficulty",
        df_group,
        groups=df_group["participant_id"]
    )
    result = model.fit()

    pval_pressure = result.pvalues["pressure[T.strong]"]
    pval_difficulty = result.pvalues["difficulty[T.hard]"]
    pval_both = result.pvalues["pressure[T.strong]:difficulty[T.hard]"]
    pvals_pressure.append(pval_pressure)
    pvals_difficulty.append(pval_difficulty)
    pvals_both.append(pval_both)
    labels.append(xai_group)

    print(result.summary())


def show_holm_results(pvals):
    reject, pvals_holm, _, _ = multipletests(
        pvals,
        alpha=0.05,
        method="holm"
    )

    for g, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
        print(
            f"{g:12s}  raw p = {p_raw:.4f}  "
            f"Holm p = {p_adj:.4f}  "
            f"{'SIGNIFICANT' if sig else 'ns'}"
        )


print("Pressure")
show_holm_results(pvals_pressure)

print("Difficulty")
show_holm_results(pvals_difficulty)

print("Both")
show_holm_results(pvals_both)


H
                     Mixed Linear Model Regression Results
Model:                      MixedLM         Dependent Variable:         reliance
No. Observations:           340             Method:                     REML    
No. Groups:                 85              Scale:                      0.0130  
Min. group size:            4               Log-Likelihood:             223.3696
Max. group size:            4               Converged:                  Yes     
Mean group size:            4.0                                                 
--------------------------------------------------------------------------------
                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
Intercept                              0.733    0.013 55.102 0.000  0.707  0.759
pressure[T.strong]                    -0.065    0.017 -3.700 0.000 -0.099 -0.030
difficulty[T.hard]                    -0.076    

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                     Mixed Linear Model Regression Results
Model:                      MixedLM         Dependent Variable:         reliance
No. Observations:           344             Method:                     REML    
No. Groups:                 86              Scale:                      0.0119  
Min. group size:            4               Log-Likelihood:             219.8741
Max. group size:            4               Converged:                  Yes     
Mean group size:            4.0                                                 
--------------------------------------------------------------------------------
                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
Intercept                              0.793    0.014 56.535 0.000  0.765  0.820
pressure[T.strong]                     0.011    0.017  0.640 0.522 -0.022  0.043
difficulty[T.hard]                    -0.001    0.

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


* (1.b) Higher time pressure and higher difficulty increase overreliance.

In [8]:
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

pvals_pressure = []
pvals_difficulty = []
pvals_both = []
labels = []
for xai_group in ["H", "H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = measures_df[measures_df["XAI condition"] == xai_group]
    print(xai_group)
    model = smf.mixedlm(
        "overreliance ~ pressure * difficulty",
        df_group,
        groups=df_group["participant_id"]
    )
    result = model.fit()

    pval_pressure = result.pvalues["pressure[T.strong]"]
    pval_difficulty = result.pvalues["difficulty[T.hard]"]
    pval_both = result.pvalues["pressure[T.strong]:difficulty[T.hard]"]
    pvals_pressure.append(pval_pressure)
    pvals_difficulty.append(pval_difficulty)
    pvals_both.append(pval_both)
    labels.append(xai_group)

    print(result.summary())


print("Pressure")
show_holm_results(pvals_pressure)

print("Difficulty")
show_holm_results(pvals_difficulty)

print("Both")
show_holm_results(pvals_both)


H
                    Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       overreliance
No. Observations:         340           Method:                   REML        
No. Groups:               85            Scale:                    0.0661      
Min. group size:          4             Log-Likelihood:           -48.8362    
Max. group size:          4             Converged:                Yes         
Mean group size:          4.0                                                 
------------------------------------------------------------------------------
                                      Coef. Std.Err.   z   P>|z| [0.025 0.975]
------------------------------------------------------------------------------
Intercept                             0.100    0.030 3.346 0.001  0.041  0.159
pressure[T.strong]                    0.059    0.039 1.492 0.136 -0.018  0.136
difficulty[T.hard]                    0.135    0.039 3.431 0.001  0.058

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                    Mixed Linear Model Regression Results
Model:                    MixedLM       Dependent Variable:       overreliance
No. Observations:         344           Method:                   REML        
No. Groups:               86            Scale:                    0.0805      
Min. group size:          4             Log-Likelihood:           -115.3722   
Max. group size:          4             Converged:                Yes         
Mean group size:          4.0                                                 
------------------------------------------------------------------------------
                                      Coef. Std.Err.   z   P>|z| [0.025 0.975]
------------------------------------------------------------------------------
Intercept                             0.198    0.039 5.099 0.000  0.122  0.274
pressure[T.strong]                    0.215    0.043 4.971 0.000  0.130  0.300
difficulty[T.hard]                    0.233    0.043 5.374 0.000  0.148  

* (1.c) Higher time pressure and higher difficulty decrease trust.


In [9]:
df_H1c = measures_df[measures_df["XAI condition"] != "H"]

In [10]:
from statsmodels.stats.multitest import multipletests
import statsmodels.formula.api as smf

pvals_pressure = []
pvals_difficulty = []
pvals_both = []
labels = []
for xai_group in ["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = df_H1c[df_H1c["XAI condition"] == xai_group]
    print(xai_group)
    model = smf.mixedlm(
        "trust ~ pressure * difficulty",
        df_group,
        groups=df_group["participant_id"]
    )
    result = model.fit()

    pval_pressure = result.pvalues["pressure[T.strong]"]
    pval_difficulty = result.pvalues["difficulty[T.hard]"]
    pval_both = result.pvalues["pressure[T.strong]:difficulty[T.hard]"]
    pvals_pressure.append(pval_pressure)
    pvals_difficulty.append(pval_difficulty)
    pvals_both.append(pval_both)
    labels.append(xai_group)

    print(result.summary())

print("Pressure")
show_holm_results(pvals_pressure)

print("Difficulty")
show_holm_results(pvals_difficulty)

print("Both")
show_holm_results(pvals_both)


H+AI
                     Mixed Linear Model Regression Results
Model:                     MixedLM         Dependent Variable:         trust    
No. Observations:          348             Method:                     REML     
No. Groups:                87              Scale:                      0.8798   
Min. group size:           4               Log-Likelihood:             -560.1614
Max. group size:           4               Converged:                  Yes      
Mean group size:           4.0                                                  
--------------------------------------------------------------------------------
                                      Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------
Intercept                              3.000    0.161 18.642 0.000  2.685  3.315
pressure[T.strong]                     0.552    0.142  3.880 0.000  0.273  0.830
difficulty[T.hard]                     0.391 

### Hypotheses 2 (Related to the benefits of XAI techniques)

Each hypothesis is tested for every group with XAI, against the H+AI baseline.


In [11]:
df_H2 = measures_df[measures_df["XAI condition"] != "H"]
df_H2 = df_H2.rename(columns={"XAI condition": "xai_condition"})

df_H2["xai_condition"] = pd.Categorical(
    df_H2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

In [12]:
df_H2[df_H2["xai_condition"] == "H+AI"]["reliance"].describe()

count    348.000000
mean       0.831897
std        0.139033
min        0.333333
25%        0.750000
50%        0.833333
75%        0.916667
max        1.000000
Name: reliance, dtype: float64

* (a) Explanations increase reliance.

In [13]:
model = smf.mixedlm(
    "reliance ~ xai_condition",
    df_H2,
    groups=df_H2["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )


                 Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      reliance 
No. Observations:        1680         Method:                  REML     
No. Groups:              420          Scale:                   0.0128   
Min. group size:         4            Log-Likelihood:          1045.3685
Max. group size:         4            Converged:               Yes      
Mean group size:         4.0                                            
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                      0.832    0.010 82.378 0.000  0.812  0.852
xai_condition[T.H+AI+CF]      -0.045    0.015 -3.096 0.002 -0.074 -0.017
xai_condition[T.H+AI+SHAP]    -0.029    0.014 -2.031 0.042 -0.057 -0.001
xai_condition[T.H+AI+LLM]      0.029    0.014  2.028 0.043  0.001  0.

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


* (b) Explanations do not increase overreliance.

In [14]:
model = smf.mixedlm(
    "overreliance ~ xai_condition",
    df_H2,
    groups=df_H2["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )


                 Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     overreliance
No. Observations:       1680        Method:                 REML        
No. Groups:             420         Scale:                  0.1245      
Min. group size:        4           Log-Likelihood:         -810.1204   
Max. group size:        4           Converged:              Yes         
Mean group size:        4.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                      0.532    0.028 18.932 0.000  0.477  0.587
xai_condition[T.H+AI+CF]      -0.035    0.041 -0.856 0.392 -0.114  0.045
xai_condition[T.H+AI+SHAP]    -0.107    0.040 -2.691 0.007 -0.185 -0.029
xai_condition[T.H+AI+LLM]     -0.051    0.040 -1.272 0.203 -0.129  0.

* (c) Explanations increase trust.

In [15]:
model = smf.mixedlm(
    "trust ~ xai_condition",
    df_H2,
    groups=df_H2["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )


                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      trust     
No. Observations:       1680         Method:                  REML      
No. Groups:             420          Scale:                   0.8933    
Min. group size:        4            Log-Likelihood:          -2657.4272
Max. group size:        4            Converged:               Yes       
Mean group size:        4.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                      3.457    0.121 28.610 0.000  3.220  3.694
xai_condition[T.H+AI+CF]      -0.073    0.175 -0.415 0.678 -0.415  0.270
xai_condition[T.H+AI+SHAP]     0.078    0.171  0.455 0.649 -0.258  0.414
xai_condition[T.H+AI+LLM]      0.564    0.172  3.271 0.001  0.226  0.

* (d) Explanations increase performance.

In [16]:
model = smf.mixedlm(
    "score ~ xai_condition",
    df_H2,
    groups=df_H2["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )

                  Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       score    
No. Observations:       1680          Method:                   REML     
No. Groups:             420           Scale:                    0.0129   
Min. group size:        4             Log-Likelihood:           1154.0927
Max. group size:        4             Converged:                Yes      
Mean group size:        4.0                                              
-------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                      0.814    0.008 105.639 0.000  0.799  0.830
xai_condition[T.H+AI+CF]      -0.032    0.011  -2.884 0.004 -0.054 -0.010
xai_condition[T.H+AI+SHAP]     0.007    0.011   0.645 0.519 -0.014  0.028
xai_condition[T.H+AI+LLM]      0.046    0.011   4.200 0.

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


### Hypotheses 3 (Related to the cognitive load)

Each hypothesis is tested for AI against H and for every group with XAI against the H+AI baseline.

* (a) When task difficulty and time pressure are low, the provision of AI predictions increases the cognitive load; the provision of explanations further amplifies this effect.


In [17]:
df_H3a1 = measures_df[(measures_df["pressure"] == "mild") & (measures_df["difficulty"] == "easy")]
df_H3a1 = df_H3a1.rename(columns={"XAI condition": "xai_condition"})

df_H3a2 = df_H3a1[df_H3a1["xai_condition"] != "H"]
df_H3a2["xai_condition"] = pd.Categorical(
    df_H3a2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/3680726686.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3a2["xai_condition"] = pd.Categorical(


In [18]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3a1,
    groups=df_H3a1["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

pvals.append(result.pvalues["xai_condition[T.H+AI]"])
labels.append("xai_condition[T.H+AI]")

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       505          Method:                  REML      
No. Groups:             505          Scale:                   18.5054   
Min. group size:        1            Log-Likelihood:          -1622.3448
Max. group size:        1            Converged:               Yes       
Mean group size:        1.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     17.329    0.624 27.760 0.000 16.106 18.553
xai_condition[T.H+AI]         -0.789    0.803 -0.983 0.326 -2.363  0.785
xai_condition[T.H+AI+CF]       1.708    0.839  2.036 0.042  0.064  3.352
xai_condition[T.H+AI+SHAP]     1.019    0.703  1.450 0.147 -0.359  2.

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [19]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3a2,
    groups=df_H3a2["participant_id"]
)
result = model.fit()
print(result.summary())

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       420          Method:                  REML      
No. Groups:             420          Scale:                   19.3682   
Min. group size:        1            Log-Likelihood:          -1358.7177
Max. group size:        1            Converged:               Yes       
Mean group size:        1.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     16.540    0.667 24.788 0.000 15.232 17.848
xai_condition[T.H+AI+CF]       2.497    0.964  2.590 0.010  0.608  4.387
xai_condition[T.H+AI+SHAP]     1.809    0.946  1.911 0.056 -0.046  3.664
xai_condition[T.H+AI+LLM]      1.210    0.952  1.271 0.204 -0.656  3.

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


* (b) When time pressure is strong, the provision of AI predictions decreases the cognitive load; the provision of explanations further amplifies this effect.

In [20]:
df_H3b1 = measures_df[(measures_df["pressure"] == "strong")]

df_H3b1 = df_H3b1.rename(columns={"XAI condition": "xai_condition"})

df_H3b2 = df_H3b1[df_H3b1["xai_condition"] != "H"]
df_H3b2["xai_condition"] = pd.Categorical(
    df_H3b2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/3573652594.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3b2["xai_condition"] = pd.Categorical(


In [21]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3b1,
    groups=df_H3b1["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

pvals.append(result.pvalues["xai_condition[T.H+AI]"])
labels.append("xai_condition[T.H+AI]")


                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       1010         Method:                  REML      
No. Groups:             505          Scale:                   12.7221   
Min. group size:        2            Log-Likelihood:          -3131.4133
Max. group size:        2            Converged:               Yes       
Mean group size:        2.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     25.512    0.628 40.625 0.000 24.281 26.743
xai_condition[T.H+AI]         -3.316    0.883 -3.756 0.000 -5.047 -1.586
xai_condition[T.H+AI+CF]      -0.824    0.902 -0.914 0.361 -2.592  0.943
xai_condition[T.H+AI+SHAP]    -2.512    0.886 -2.836 0.005 -4.247 -0.

In [22]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3b2,
    groups=df_H3b2["participant_id"]
)
result = model.fit()
print(result.summary())

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       840          Method:                  REML      
No. Groups:             420          Scale:                   12.2119   
Min. group size:        2            Log-Likelihood:          -2601.0313
Max. group size:        2            Converged:               Yes       
Mean group size:        2.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     22.195    0.629 35.304 0.000 20.963 23.428
xai_condition[T.H+AI+CF]       2.492    0.908  2.744 0.006  0.712  4.272
xai_condition[T.H+AI+SHAP]     0.805    0.892  0.902 0.367 -0.943  2.552
xai_condition[T.H+AI+LLM]      0.888    0.897  0.990 0.322 -0.870  2.

* (c) When task difficulty is high, the provision of AI predictions decreases the cognitive load; the provision of explanations further amplifies this effect.

In [23]:
df_H3c1 = measures_df[(measures_df["difficulty"] == "hard")]
df_H3c1 = df_H3c1.rename(columns={"XAI condition": "xai_condition"})

df_H3c2 = df_H3c1[df_H3c1["xai_condition"] != "H"]
df_H3c2["xai_condition"] = pd.Categorical(
    df_H3c2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/1951546652.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3c2["xai_condition"] = pd.Categorical(


In [24]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3c1,
    groups=df_H3c1["participant_id"]
)
result = model.fit()
print(result.summary())

pvals = []
labels = []

pvals.append(result.pvalues["xai_condition[T.H+AI]"])
labels.append("xai_condition[T.H+AI]")


                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       1010         Method:                  REML      
No. Groups:             505          Scale:                   21.3148   
Min. group size:        2            Log-Likelihood:          -3233.4603
Max. group size:        2            Converged:               Yes       
Mean group size:        2.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     24.671    0.593 41.574 0.000 23.508 25.834
xai_condition[T.H+AI]         -3.067    0.834 -3.676 0.000 -4.702 -1.432
xai_condition[T.H+AI+CF]      -1.014    0.852 -1.190 0.234 -2.685  0.656
xai_condition[T.H+AI+SHAP]    -2.566    0.837 -3.066 0.002 -4.206 -0.

In [25]:
model = smf.mixedlm(
    "cogload ~ xai_condition",
    df_H3c2,
    groups=df_H3c2["participant_id"]
)
result = model.fit()
print(result.summary())

for term in result.pvalues.index:
    if term.startswith("xai_condition[T."):
        pvals.append(result.pvalues[term])
        labels.append(term)

reject, pvals_holm, _, _ = multipletests(
    pvals,
    alpha=0.05,
    method="holm"
)

for lab, p_raw, p_adj, sig in zip(labels, pvals, pvals_holm, reject):
    print(
        f"{lab:30s} raw p = {p_raw:.4f}, "
        f"Holm p = {p_adj:.4f}, "
        f"{'SIGNIFICANT' if sig else 'ns'}"
    )

                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      cogload   
No. Observations:       840          Method:                  REML      
No. Groups:             420          Scale:                   20.0834   
Min. group size:        2            Log-Likelihood:          -2680.0988
Max. group size:        2            Converged:               Yes       
Mean group size:        2.0                                             
------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------
Intercept                     21.603    0.591 36.531 0.000 20.444 22.763
xai_condition[T.H+AI+CF]       2.053    0.854  2.403 0.016  0.378  3.727
xai_condition[T.H+AI+SHAP]     0.501    0.839  0.598 0.550 -1.143  2.145
xai_condition[T.H+AI+LLM]      0.212    0.844  0.251 0.802 -1.442  1.

In [26]:
def show_bayesian_results(
        posteriors,
        labels,
        direction="positive",
        threshold=0.95,
        rope=None,
        delta=None
):
    """
    Display Bayesian results for multiple hypotheses.

    Parameters
    ----------
    posteriors : list of np.array
        Posterior samples for each condition.
    labels : list of str
        Names of the conditions.
    direction : str
        'positive'   → test effect > 0
        'negative'   → test effect < 0
        'rope'       → test |effect| in ROPE (no meaningful effect)
        'no_increase'→ test effect does not increase overreliance (effect ≤ delta)
    threshold : float
        Probability threshold for “strong evidence”
    rope : tuple
        (lower, upper) bounds for ROPE, required if direction='rope'
    delta : float
        Threshold for meaningful increase, required if direction='no_increase'
    """

    print()
    for lab, samples in zip(labels, posteriors):

        if direction == "positive":
            p = (samples > 0).mean()
            sig = p >= threshold
            print(f"{lab:12s}  P(effect > 0) = {p:.3f}  "
                  f"{'STRONG EVIDENCE' if sig else 'weak / inconclusive'}")

        elif direction == "negative":
            p = (samples < 0).mean()
            sig = p >= threshold
            print(f"{lab:12s}  P(effect < 0) = {p:.3f}  "
                  f"{'STRONG EVIDENCE' if sig else 'weak / inconclusive'}")

        elif direction == "rope":
            if rope is None:
                raise ValueError("ROPE bounds must be provided for 'rope' test")
            lower, upper = rope
            p = ((samples > lower) & (samples < upper)).mean()
            sig = p >= threshold
            print(f"{lab:12s}  P(|effect| in [{lower}, {upper}]) = {p:.3f}  "
                  f"{'NO EFFECT SUPPORTED' if sig else 'possible effect'}")

        elif direction == "no_increase":
            if delta is None:
                raise ValueError("delta must be provided for 'no_increase' test")
            p = (samples > delta).mean()   # posterior probability of meaningful increase
            sig = p <= (1 - threshold)     # small probability supports hypothesis
            print(f"{lab:12s}  P(effect > {delta}) = {p:.3f}  "
                  f"{'NO INCREASE SUPPORTED' if sig else 'possible increase'}")

        else:
            raise ValueError("direction must be 'positive', 'negative', 'rope', or 'no_increase'")


# Basyesian analysis

## Test of hypotheses

### Hypotheses 1 (related to time pressure)
Each hypothesis is tested for every group with AI, against the data of the same group
with no time pressure/difficulty. Effects of pressure and
difficulty are studied independently + their interaction

* (1.a) Higher time pressure and higher difficulty increase reliance.

In [27]:
import bambi as bmb

posterior_pressure = []
posterior_difficulty = []
posterior_both = []
labels = []

for xai_group in ["H", "H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = measures_df[measures_df["XAI condition"] == xai_group]
    df_group = df_group.drop(columns=["tasks_order"])
    print(xai_group)

    # Fit Bayesian mixed model
    model = bmb.Model(
        "reliance ~ pressure * difficulty + (1|participant_id)",
        df_group,
        family="gaussian"
    )

    trace = model.fit(
        draws=4000,
        tune=2000,
        target_accept=0.95
    )

    post = trace.posterior  # xarray Dataset

    # Extract posterior samples for each effect as numpy arrays
    posterior_pressure.append(post["pressure"].values.flatten())
    posterior_difficulty.append(post["difficulty"].values.flatten())
    posterior_both.append(post["pressure:difficulty"].values.flatten())
    labels.append(xai_group)

# Now display results using the latest show_bayesian_results

print("Pressure")
show_bayesian_results(
    posterior_pressure,
    labels
)

print("Difficulty")
show_bayesian_results(
    posterior_difficulty,
    labels
)

print("Both")
show_bayesian_results(
    posterior_both,
    labels,
)


H


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 22 seconds.


H+AI


...

H+AI+GRADCAM


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 20 seconds.


Pressure

H             P(effect > 0) = 0.000  weak / inconclusive
H+AI          P(effect > 0) = 0.812  weak / inconclusive
H+AI+CF       P(effect > 0) = 0.898  weak / inconclusive
H+AI+SHAP     P(effect > 0) = 0.737  weak / inconclusive
H+AI+LLM      P(effect > 0) = 0.938  weak / inconclusive
H+AI+GRADCAM  P(effect > 0) = 0.574  weak / inconclusive
Difficulty

H             P(effect > 0) = 0.000  weak / inconclusive
H+AI          P(effect > 0) = 0.380  weak / inconclusive
H+AI+CF       P(effect > 0) = 0.169  weak / inconclusive
H+AI+SHAP     P(effect > 0) = 0.479  weak / inconclusive
H+AI+LLM      P(effect > 0) = 0.985  STRONG EVIDENCE
H+AI+GRADCAM  P(effect > 0) = 0.123  weak / inconclusive
Both

H             P(effect > 0) = 0.004  weak / inconclusive
H+AI          P(effect > 0) = 0.980  STRONG EVIDENCE
H+AI+CF       P(effect > 0) = 0.816  weak / inconclusive
H+AI+SHAP     P(effect > 0) = 0.818  weak / inconclusive
H+AI+LLM      P(effect > 0) = 0.062  weak / inconclusive
H+AI+GRADCA

* (1.b) Higher time pressure and higher difficulty increase overreliance.

In [28]:
import bambi as bmb

posterior_pressure = []
posterior_difficulty = []
posterior_both = []
labels = []

for xai_group in ["H", "H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = measures_df[measures_df["XAI condition"] == xai_group]
    df_group = df_group.drop(columns=["tasks_order"])
    print(xai_group)

    # Fit Bayesian mixed model
    model = bmb.Model(
        "overreliance ~ pressure * difficulty + (1|participant_id)",
        df_group,
        family="gaussian"
    )

    trace = model.fit(
        draws=4000,
        tune=2000,
        target_accept=0.95
    )

    post = trace.posterior  # xarray Dataset

    # Extract posterior samples for each effect as numpy arrays
    posterior_pressure.append(post["pressure"].values.flatten())
    posterior_difficulty.append(post["difficulty"].values.flatten())
    posterior_both.append(post["pressure:difficulty"].values.flatten())
    labels.append(xai_group)

# Now display results using the latest show_bayesian_results

print("Pressure")
show_bayesian_results(
    posterior_pressure,
    labels
)

print("Difficulty")
show_bayesian_results(
    posterior_difficulty,
    labels
)

print("Both")
show_bayesian_results(
    posterior_both,
    labels,
)


H


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 20 seconds.


H+AI


...

H+AI+GRADCAM


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 19 seconds.


Pressure

H             P(effect > 0) = 0.927  weak / inconclusive
H+AI          P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+CF       P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+SHAP     P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+LLM      P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+GRADCAM  P(effect > 0) = 1.000  STRONG EVIDENCE
Difficulty

H             P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI          P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+CF       P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+SHAP     P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+LLM      P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+GRADCAM  P(effect > 0) = 0.999  STRONG EVIDENCE
Both

H             P(effect > 0) = 0.995  STRONG EVIDENCE
H+AI          P(effect > 0) = 0.799  weak / inconclusive
H+AI+CF       P(effect > 0) = 0.711  weak / inconclusive
H+AI+SHAP     P(effect > 0) = 0.574  weak / inconclusive
H+AI+LLM      P(effect > 0) = 0.474  weak / inconclusive
H+AI+GRADCAM  P(effect > 0) = 0.829  weak / inconcl

* (1.c) Higher time pressure and higher difficulty decrease trust.

In [30]:
df_H1c = measures_df[measures_df["XAI condition"] != "H"]

In [31]:
import bambi as bmb

posterior_pressure = []
posterior_difficulty = []
posterior_both = []
labels = []

for xai_group in ["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"]:
    df_group = df_H1c[df_H1c["XAI condition"] == xai_group]
    df_group = df_group.drop(columns=["tasks_order"])
    print(xai_group)

    # Fit Bayesian mixed model
    model = bmb.Model(
        "trust ~ pressure * difficulty + (1|participant_id)",
        df_group,
        family="gaussian"
    )

    trace = model.fit(
        draws=4000,
        tune=2000,
        target_accept=0.95
    )

    post = trace.posterior  # xarray Dataset

    # Extract posterior samples for each effect as numpy arrays
    posterior_pressure.append(post["pressure"].values.flatten())
    posterior_difficulty.append(post["difficulty"].values.flatten())
    posterior_both.append(post["pressure:difficulty"].values.flatten())
    labels.append(xai_group)

# Now display results using the latest show_bayesian_results

print("Pressure")
show_bayesian_results(
    posterior_pressure,
    labels,
    direction="negative"
)

print("Difficulty")
show_bayesian_results(
    posterior_difficulty,
    labels,
    direction="negative"
)

print("Both")
show_bayesian_results(
    posterior_both,
    labels,
    direction="negative",
)


Initializing NUTS using jitter+adapt_diag...


H+AI


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 20 seconds.
Initializing NUTS using jitter+adapt_diag...


...

H+AI+GRADCAM


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, pressure, difficulty, pressure:difficulty, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 18 seconds.


Pressure

H+AI          P(effect < 0) = 0.000  weak / inconclusive
H+AI+CF       P(effect < 0) = 0.186  weak / inconclusive
H+AI+SHAP     P(effect < 0) = 0.027  weak / inconclusive
H+AI+LLM      P(effect < 0) = 0.049  weak / inconclusive
H+AI+GRADCAM  P(effect < 0) = 0.315  weak / inconclusive
Difficulty

H+AI          P(effect < 0) = 0.003  weak / inconclusive
H+AI+CF       P(effect < 0) = 0.539  weak / inconclusive
H+AI+SHAP     P(effect < 0) = 0.525  weak / inconclusive
H+AI+LLM      P(effect < 0) = 0.194  weak / inconclusive
H+AI+GRADCAM  P(effect < 0) = 0.744  weak / inconclusive
Both

H+AI          P(effect < 0) = 0.609  weak / inconclusive
H+AI+CF       P(effect < 0) = 0.011  weak / inconclusive
H+AI+SHAP     P(effect < 0) = 0.017  weak / inconclusive
H+AI+LLM      P(effect < 0) = 0.318  weak / inconclusive
H+AI+GRADCAM  P(effect < 0) = 0.189  weak / inconclusive


### Hypotheses 2 (Related to the benefits of XAI techniques)

Each hypothesis is tested for every group with XAI, against the H+AI baseline.

In [32]:
df_H2 = measures_df[measures_df["XAI condition"] != "H"]
df_H2 = df_H2.rename(columns={"XAI condition": "xai_condition"})
df_H2 = df_H2.drop(columns=["tasks_order"])


df_H2["xai_condition"] = pd.Categorical(
    df_H2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

* (a) Explanations increase reliance.

In [33]:
model = bmb.Model(
    "reliance ~ xai_condition + (1|participant_id)",
    df_H2,
    family="gaussian"
)

trace = model.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post = trace.posterior["xai_condition"]
labels = post.coords["xai_condition_dim"].values.tolist()

posterior_samples = [
    post.isel(xai_condition_dim=i).values.flatten()
    for i in range(len(labels))
]

show_bayesian_results(posterior_samples, labels)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 89 seconds.



H+AI+CF       P(effect > 0) = 0.001  weak / inconclusive
H+AI+GRADCAM  P(effect > 0) = 0.004  weak / inconclusive
H+AI+LLM      P(effect > 0) = 0.979  STRONG EVIDENCE
H+AI+SHAP     P(effect > 0) = 0.022  weak / inconclusive


* (b) Explanations do not increase overreliance

In [34]:
model = bmb.Model(
    "overreliance ~ xai_condition + (1|participant_id)",
    df_H2,
    family="gaussian"
)

delta = 0.001

trace = model.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post = trace.posterior["xai_condition"]
labels = post.coords["xai_condition_dim"].values.tolist()

posterior_samples = [
    post.isel(xai_condition_dim=i).values.flatten()
    for i in range(len(labels))
]

show_bayesian_results(posterior_samples, labels, direction="no_increase", delta=delta, threshold=0.95)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 91 seconds.



H+AI+CF       P(effect > 0.001) = 0.188  possible increase
H+AI+GRADCAM  P(effect > 0.001) = 0.001  NO INCREASE SUPPORTED
H+AI+LLM      P(effect > 0.001) = 0.098  possible increase
H+AI+SHAP     P(effect > 0.001) = 0.004  NO INCREASE SUPPORTED


* (c) Explanations increase trust.

In [35]:
model = bmb.Model(
    "trust ~ xai_condition + (1|participant_id)",
    df_H2,
    family="gaussian"
)

trace = model.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post = trace.posterior["xai_condition"]
labels = post.coords["xai_condition_dim"].values.tolist()

posterior_samples = [
    post.isel(xai_condition_dim=i).values.flatten()
    for i in range(len(labels))
]

show_bayesian_results(posterior_samples, labels)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 85 seconds.



H+AI+CF       P(effect > 0) = 0.338  weak / inconclusive
H+AI+GRADCAM  P(effect > 0) = 0.180  weak / inconclusive
H+AI+LLM      P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+SHAP     P(effect > 0) = 0.670  weak / inconclusive


* (d) Explanations increase performance

In [36]:
model = bmb.Model(
    "score ~ xai_condition + (1|participant_id)",
    df_H2,
    family="gaussian"
)

trace = model.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post = trace.posterior["xai_condition"]
labels = post.coords["xai_condition_dim"].values.tolist()

posterior_samples = [
    post.isel(xai_condition_dim=i).values.flatten()
    for i in range(len(labels))
]

show_bayesian_results(posterior_samples, labels)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 92 seconds.



H+AI+CF       P(effect > 0) = 0.002  weak / inconclusive
H+AI+GRADCAM  P(effect > 0) = 0.828  weak / inconclusive
H+AI+LLM      P(effect > 0) = 1.000  STRONG EVIDENCE
H+AI+SHAP     P(effect > 0) = 0.741  weak / inconclusive


### Hypotheses 3 (Related to the cognitive load)

Each hypothesis is tested for AI against H and for every group with XAI against the H+AI baseline.

* (a) When task difficulty and time pressure are low, the provision of AI predictions increases the cognitive load; the provision of explanations further amplifies this effect.


In [37]:
df_H3a1 = measures_df[(measures_df["pressure"] == "mild") & (measures_df["difficulty"] == "easy")]
df_H3a1 = df_H3a1.rename(columns={"XAI condition": "xai_condition"})
df_H3a1 = df_H3a1.drop(columns=["tasks_order"])

df_H3a2 = df_H3a1[df_H3a1["xai_condition"] != "H"]
df_H3a2["xai_condition"] = pd.Categorical(
    df_H3a2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/1088545782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3a2["xai_condition"] = pd.Categorical(


In [38]:
import bambi as bmb

posterior_probs = []
labels = []

# -------------------------
# First dataset: H3a1
# -------------------------
model1 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3a1,
    family="gaussian"
)

trace1 = model1.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post1 = trace1.posterior["xai_condition"]

# Example: testing a single condition (H+AI)
labels.append("xai_condition[T.H+AI]")
posterior_probs.append(post1.sel(xai_condition_dim="H+AI").values.flatten())

# -------------------------
# Second dataset: H3a2
# -------------------------
model2 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3a2,
    family="gaussian"
)

trace2 = model2.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post2 = trace2.posterior["xai_condition"]

# Iterate over all conditions vs reference
for cond in post2.coords["xai_condition_dim"].values:
    labels.append(f"xai_condition[T.{cond}]")
    posterior_probs.append(post2.sel(xai_condition_dim=cond).values.flatten())


# Example: test for a positive effect
show_bayesian_results(
    posterior_probs,
    labels,
    direction="positive",
    threshold=0.95,
)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 48 seconds.
There were 1333 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 37 seconds.
There were 2188 divergences after tuning. Increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details



xai_condition[T.H+AI]  P(effect > 0) = 0.188  weak / inconclusive
xai_condition[T.H+AI+CF]  P(effect > 0) = 0.996  STRONG EVIDENCE
xai_condition[T.H+AI+GRADCAM]  P(effect > 0) = 0.872  weak / inconclusive
xai_condition[T.H+AI+LLM]  P(effect > 0) = 0.885  weak / inconclusive
xai_condition[T.H+AI+SHAP]  P(effect > 0) = 0.969  STRONG EVIDENCE


* (b) When time pressure is strong, the provision of AI predictions decreases the cognitive load; the provision of explanations further amplifies this effect.


In [39]:
df_H3b1 = measures_df[(measures_df["pressure"] == "strong")]
df_H3b1 = df_H3b1.rename(columns={"XAI condition": "xai_condition"})
df_H3b1 = df_H3b1.drop(columns=["tasks_order"])

df_H3b2 = df_H3b1[df_H3b1["xai_condition"] != "H"]
df_H3b2["xai_condition"] = pd.Categorical(
    df_H3b2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/296778882.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3b2["xai_condition"] = pd.Categorical(


In [40]:
import bambi as bmb

posterior_probs = []
labels = []

# -------------------------
# First dataset: H3a1
# -------------------------
model1 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3b1,
    family="gaussian"
)

trace1 = model1.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post1 = trace1.posterior["xai_condition"]

# Example: testing a single condition (H+AI)
labels.append("xai_condition[T.H+AI]")
posterior_probs.append(post1.sel(xai_condition_dim="H+AI").values.flatten())

# -------------------------
# Second dataset: H3a2
# -------------------------
model2 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3b2,
    family="gaussian"
)

trace2 = model2.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post2 = trace2.posterior["xai_condition"]

# Iterate over all conditions vs reference
for cond in post2.coords["xai_condition_dim"].values:
    labels.append(f"xai_condition[T.{cond}]")
    posterior_probs.append(post2.sel(xai_condition_dim=cond).values.flatten())


# Example: test for a positive effect
show_bayesian_results(
    posterior_probs,
    labels,
    direction="negative",
    threshold=0.95,
)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 76 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 32 seconds.



xai_condition[T.H+AI]  P(effect < 0) = 1.000  STRONG EVIDENCE
xai_condition[T.H+AI+CF]  P(effect < 0) = 0.004  weak / inconclusive
xai_condition[T.H+AI+GRADCAM]  P(effect < 0) = 0.060  weak / inconclusive
xai_condition[T.H+AI+LLM]  P(effect < 0) = 0.160  weak / inconclusive
xai_condition[T.H+AI+SHAP]  P(effect < 0) = 0.187  weak / inconclusive


* (c) When task difficulty is high, the provision of AI predictions decreases the cognitive load; the provision of explanations further amplifies this effect.


In [42]:
df_H3c1 = measures_df[(measures_df["difficulty"] == "hard")]
df_H3c1 = df_H3c1.rename(columns={"XAI condition": "xai_condition"})
df_H3c1 = df_H3c1.drop(columns=["tasks_order"])

df_H3c2 = df_H3c1[df_H3c1["xai_condition"] != "H"]
df_H3c2["xai_condition"] = pd.Categorical(
    df_H3c2["xai_condition"],
    categories=["H+AI", "H+AI+CF", "H+AI+SHAP", "H+AI+LLM", "H+AI+GRADCAM"],
    ordered=False
)

/tmp/ipykernel_3524739/1384528324.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_H3c2["xai_condition"] = pd.Categorical(


In [43]:
import bambi as bmb

posterior_probs = []
labels = []

# -------------------------
# First dataset: H3a1
# -------------------------
model1 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3c1,
    family="gaussian"
)

trace1 = model1.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post1 = trace1.posterior["xai_condition"]

# Example: testing a single condition (H+AI)
labels.append("xai_condition[T.H+AI]")
posterior_probs.append(post1.sel(xai_condition_dim="H+AI").values.flatten())

# -------------------------
# Second dataset: H3a2
# -------------------------
model2 = bmb.Model(
    "cogload ~ xai_condition + (1|participant_id)",
    df_H3c2,
    family="gaussian"
)

trace2 = model2.fit(
    draws=4000,
    tune=2000,
    target_accept=0.95
)

post2 = trace2.posterior["xai_condition"]

# Iterate over all conditions vs reference
for cond in post2.coords["xai_condition_dim"].values:
    labels.append(f"xai_condition[T.{cond}]")
    posterior_probs.append(post2.sel(xai_condition_dim=cond).values.flatten())


# Example: test for a positive effect
show_bayesian_results(
    posterior_probs,
    labels,
    direction="negative",
    threshold=0.95,
)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 78 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, xai_condition, 1|participant_id_sigma, 1|participant_id_offset]


Output()

Sampling 4 chains for 2_000 tune and 4_000 draw iterations (8_000 + 16_000 draws total) took 33 seconds.



xai_condition[T.H+AI]  P(effect < 0) = 1.000  STRONG EVIDENCE
xai_condition[T.H+AI+CF]  P(effect < 0) = 0.009  weak / inconclusive
xai_condition[T.H+AI+GRADCAM]  P(effect < 0) = 0.180  weak / inconclusive
xai_condition[T.H+AI+LLM]  P(effect < 0) = 0.395  weak / inconclusive
xai_condition[T.H+AI+SHAP]  P(effect < 0) = 0.278  weak / inconclusive
